# Creating a MOCK dataset 
Author: Karina Condeixa
__________________________________________

**[WIP]**

The process to create a fake dataset started by making dataframes about `users` and `item`, and further a complete dataframe that could be used in the `model`.

`numpy random` and `faker` were used to ccreate randomized series. Latitude_longitude points were picked using a polygons method, consideing four points choosen manually in Google Maps.

Dependent variables are part of a same series, identified by underscore.

Addresses are illustrative, they are addresses from all over Germany. This is a limitation of the Faker package, which only considers countries, and does not have data for Berlin.

_________________________________

**TODO:**
- create final dataset, organising by columns **[check with Larry and Felipe]**
- save the files as csv

**Coments:** 
- coordinates of user who post should became the ones for available item
- we don't have info about item's type and category in the app, could be needed to parse posts' titles and description
- UXs didn't decide at this point about having number of stars to a post and the classifications in conditions. I will adopt 1,2,3 for conditions and add the number of starts in next steps. For condition, 1 is the best and 3 the worse. For stars, 5 is the best and 1 is the worse

In [12]:
# import packages
import pandas as pd
from faker import Faker, providers
from faker.providers.address.de_DE import Provider as DeDeAddressProvider
from faker.generator import random
from faker.providers import BaseProvider
# import random

import folium
# from geopy.geocoders import Nominatim

### Postcodes

In [13]:
# immport and clean original dataset removing poostcodes from Germanay, that are not in Berlin
postcodes_de = pd.read_excel(r'data/German-Zip-Codes.xlsx', sheet_name='Berlin')
df = pd.DataFrame(postcodes_de)
df.set_axis(["postcodes_berlin"], axis=1,inplace=True)
df = (df["postcodes_berlin"].str[8:-11])
df.to_csv('data/postcodes_berlin.csv', index=False)

/var/folders/xs/pmxwbppj14b4v76l4llp17n80000gp/T/ipykernel_85728/2575223029.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(["postcodes_berlin"], axis=1,inplace=True)


### Creating the postcode series

In [14]:
# import postcodes from Berlin and create a dataframe removing indexes and headers
postcodes_berlin = pd.read_csv('data/postcodes_berlin.csv')
print(postcodes_berlin)
postcodes_berlin_series = postcodes_berlin[:][1:].squeeze()

     postcodes_berlin
0               10117
1               10119
2               10178
3               10179
4               10243
..                ...
184             14169
185             14193
186             14195
187             14197
188             14199

[189 rows x 1 columns]


### Creating lists

#### lat lng : choosing random point based on a polygon
reference: 
    A quick trick to create random lat/long coordinates in python (within a defined polygon)
https://medium.com/the-data-journal/a-quick-trick-to-create-random-lat-long-coordinates-in-python-within-a-defined-polygon-e8997f05123a

In [15]:
# Importing Modules
import numpy as np
import random
# Use 'conda install shapely' to import the shapely library.
from shapely.geometry import Polygon, Point

num_records = 10

### Creating the datasets

In [16]:
# Note: # multi_locale_generator = Faker(['it_IT', 'en_US', 'de-DE', 'pt_BR', 'es-ES', 'fr-FR', 'ru-RU', 'tr-TR'])

# Instantiate Faker with multiple locales
german_locale_generator = Faker(['de_DE'])
fake = Faker()
Faker.seed(0)


### Using polygons [DEPRECATED]

In [17]:
# Define the desired polygon : points choosen in Google maps

poly = Polygon([(52.645883, 13.395869), 
                (52.526568, 13.645808),
                (52.381789, 13.405482),
                (52.484773, 13.136317)])


min_x = 52.381789
max_x = 52.645883
min_y = 13.136317
max_y = 13.645808

# Defining the randomization generator
def polygon_random_points (poly, num_records):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_records:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)
    return points
    # Choose the number of points desired. T\ 
points = polygon_random_points(poly,num_records)
# Testing the results.
for p in points:
    print(p.x,",",p.y)

                             

52.55680098788801 , 13.28041659882159
52.550807097445094 , 13.270587138273807
52.529707777750644 , 13.282135730490767
52.480690625298884 , 13.396928281959612
52.57230040032704 , 13.470134155052126
52.4555519775352 , 13.273365460155922
52.50392258137691 , 13.577237302021752
52.52407107636504 , 13.367777338068155
52.54478247099107 , 13.389837216340199
52.560255762116896 , 13.478321437688123


#### The polygon in the map

In [18]:
mapObj = folium.Map(location=[52.520008,13.404954], zoom_start=5)

In [19]:
folium.Polygon([(52.645883, 13.395869), 
                (52.526568, 13.645808), 
                (52.381789, 13.405482), 
                (52.484773, 13.136317)],
                fill=True, 
                weight=3, 
                fill_color='orange',
                color='green',
                fill_opacity=0.8).add_to(mapObj)

mapObj.save('data/output.html')

### List of latitude and longitude co-ordinates 
Based on a sample of co-ordinates from [free data Berlin](https://daten.berlin.de/datensaetze/stra%C3%9Fenverkehrsunf%C3%A4lle-nach-unfallort-berlin-2021) for Road traffic accidents by accident location in Berlin 2021.


In [22]:
lat_lng_samples = pd.read_csv("data/lat_lng_samples.csv")
lat_lng_samples.head(2)

,lat,lng
0,"5,253,393,955","1,342,689,483"
1,"5,243,995,086","1,339,209,027"


In [23]:
# replace comma to dot
lat_lng_samples = lat_lng_samples.apply(lambda x: x.str.replace(',','.'))

In [24]:
list_lat = lat_lng_samples.lat.values.tolist()
list_lng = lat_lng_samples.lng.values.tolist()
lat_lng_samples.head(2)


,lat,lng
0,5.253.393.955,1.342.689.483
1,5.243.995.086,1.339.209.027


In [25]:
lat_lng_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11197 entries, 0 to 11196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     11192 non-null  object
 1   lng     11192 non-null  object
dtypes: object(2)
memory usage: 175.1+ KB


### User

In [26]:
# define a function to create user data

user_type = ['giver', 'looker']


def create_user_data(num_records): 
  
    # dictionary 
    user ={} 
    for i in range(0, num_records): 
        user[i] = {} 
        user[i]['name'] = fake.name()
#         user[i]['email'] = fake.email()
#         user[i]['email'] = fake.ascii_free_email()
        user[i]['email'] = fake.ascii_email()
        address = german_locale_generator.address()  # these addresses are from germany, find a list of address for berlin
        user[i]['address'] = address
        user[i]['user_type'] = fake.random_element(user_type)
        user[i]['user_lat'] = np.random.choice(list_lat)
        user[i]['user_lng'] = np.random.choice(list_lng)
        user[i]['user_postcode'] = np.random.choice(postcodes_berlin_series)

    return user

In [27]:
user_mock_df = pd.DataFrame(create_user_data(1500)).transpose()
user_mock_df.head(5)

,name,email,address,user_type,user_lat,user_lng,user_postcode
0,Norma Fisher,ysullivan@yahoo.com,Alwina-Etzold-Ring 19\n89241 Sömmerda,looker,5.249.130.415,1.362.630.686,14059
1,Brian Hamilton,hramos@brown-sellers.com,Eleonore-Oderwald-Ring 51\n93328 Bremen,giver,525.029.349,1.347.343.365,10961
2,Sheri Bolton DDS,jasmine85@hotmail.com,Conradistr. 5/9\n42320 Naila,giver,5.253.511.142,1.326.626.049,12209
3,Peter Mcdowell,villanuevasandra@vega.net,Salzring 7/5\n59179 Erfurt,giver,5.254.529.961,1.337.752.135,12159
4,Devin Thornton,marvincabrera@gmail.com,Margot-Ruppert-Allee 013\n61510 Euskirchen,giver,5.256.141.258,1.339.519.243,12057


In [28]:
user_mock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           1500 non-null   object
 1   email          1500 non-null   object
 2   address        1500 non-null   object
 3   user_type      1500 non-null   object
 4   user_lat       1500 non-null   object
 5   user_lng       1500 non-null   object
 6   user_postcode  1500 non-null   object
dtypes: object(7)
memory usage: 93.8+ KB


In [29]:
# add user_ids 
user_mock_df['user_id'] = user_mock_df.index + 1
user_id_series = user_mock_df['user_id']

### Item

In [30]:
item_status = ['avaliable', 'not_available']
item_condition = [1,2,3]  # ['good','medium','bad']
n_likes = [1, 2, 3, 4, 5]  # 1 is worse, 5 is best

category_item = ['furniture-sofa',
                 'furniture-armchair',
                 'furniture-chair',
                 'furniture-table',
                 'furniture-desk',
                 'furniture-bed',
                 'furniture-bookcase',
                 'furniture-bedside_table',
                 'furniture-cabinet',
                 'furniture-wardrobe',
                 'furniture-shelf',
                 'furniture-cupboard',
                 'furniture-rollcontainers',
                 'furniture-shoe_rack',
                 'furniture-mirror',
                 'furniture-cot',
                 'furniture-trolley',
                 'appliance-washing_machine',
                 'appliance-dish_washer',
                 'appliance-drying_rack',
                 'appliance-refrigerator',
                 'appliance-blender',
                 'appliance-extractor_hood',
                 'appliance-clothes_iron',
                 'appliance-vacuum_cleaner',
                 'appliance-sandwich_maker',
                 'appliance-kettle',
                 'appliance-air_conditioner',
                 'appliance-heater',
                 'appliance-pan',
                 'appliance-popcorn_maker',
                 'appliance-coffee_machine',
                 'appliance-stove',
                 'lighting-lighting',
                 'lighting-chandelier',
                 'lighting-lightbulb',
                 'musical_equipment-guitar',
                 'musical_equipment-sound_amplifier',
                 'musical_equipment-contrabass',
                 'musical_equipment-battery',
                 'musical_equipment-piano',
                 'tech-desktop',
                 'tech-laptop',
                 'tech-phone',
                 'tech-keyboard',
                 'clothes-woman_jacket',
                 'clothes-man_jacket',
                 'clothes-child_jacket',
                 'clothes-woman_clothes',
                 'clothes-man_clothes',
                 'clothes-child_clothes',
                 'shoes-woman_shoes',
                 'shoes-man_shoes',
                 'shoes-child_shoes',
                 'miscelaneaous-ironing_board',
                 'miscelaneaous-picture_frame',
                 'miscelaneaous-bicycle',
                 'miscelaneaous-plant',
                 'miscelaneaous-carpet',
                 'miscelaneaous-roller_skates',
                 'miscelaneaous-ski_skates',
                 'miscelaneaous-books',
                 'miscelaneaous-purse',
                 'miscelaneaous-suitcase',
                 'miscelaneaous-shopping_venture',
                 'miscelaneaous-board',
                 'miscelaneaous-frame',
                 'home-mattress', 
                 'home-carpet',
                 'kids-stroller',
                 'kids-baby_carriage']


In [31]:
# define a function to create item data
limit = '-30d'  # limit of 30 days of item in the app

def create_item_data(num_records): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records): 
        item[i] = {}
        item[i]['item_category-item_name'] = np.random.choice(category_item)
        item[i]['item_condition'] = np.random.choice(item_condition)
        item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
        item[i]['item_status'] = np.random.choice(item_status)
        
        item[i]['item_lat'] = np.random.choice(list_lat)
        item[i]['item_lng'] = np.random.choice(list_lng)
        
        item[i]['userwhochangeditemstatus_id'] = np.random.choice(user_id_series)
        datetime_iteration1 = fake.date_between_dates(limit,'now')
        datetime_iteration2 = fake.date_between_dates(limit,'now')
        if datetime_iteration1 <= datetime_iteration2:
            item[i]['item_datetime_posted'] = datetime_iteration1
            item[i]['item_datetimechangeditemstatus'] = datetime_iteration2
        else:
            item[i]['item_datetime_posted'] = datetime_iteration2
            item[i]['item_datetimechangeditemstatus'] = datetime_iteration1  
 # This date shold be later than the post
        

    return item

In [32]:
item_mock_df = pd.DataFrame(create_item_data(1500)).transpose()
item_mock_df.head(5)

,item_category-item_name,item_condition,item_postcode,item_status,item_lat,item_lng,userwhochangeditemstatus_id,item_datetime_posted,item_datetimechangeditemstatus
0,furniture-shelf,2,13057,not_available,5.241.665.604,1.359.760.045,317,2023-02-20,2023-03-08
1,kids-baby_carriage,1,13158,avaliable,5.253.623.803,1.336.467.355,770,2023-02-27,2023-03-09
2,tech-laptop,3,10587,avaliable,5.253.322.568,1.355.167.911,1450,2023-03-04,2023-03-13
3,appliance-drying_rack,1,12524,not_available,5.255.570.389,1.343.193.747,276,2023-03-03,2023-03-04
4,miscelaneaous-books,1,10367,not_available,5.244.443.575,1.334.066.855,1377,2023-03-05,2023-03-16


In [33]:
item_mock_df['item_id'] = item_mock_df.index +1  # add item_id
item_id_series = item_mock_df['item_id']  # storage in a variable to use later

item_mock_df.head(2)

,item_category-item_name,item_condition,item_postcode,item_status,item_lat,item_lng,userwhochangeditemstatus_id,item_datetime_posted,item_datetimechangeditemstatus,item_id
0,furniture-shelf,2,13057,not_available,5.241.665.604,1.359.760.045,317,2023-02-20,2023-03-08,1
1,kids-baby_carriage,1,13158,avaliable,5.253.623.803,1.336.467.355,770,2023-02-27,2023-03-09,2


In [34]:
item_mock_df[['item_category','item_name']] = item_mock_df['item_category-item_name'].apply(lambda x: pd.Series(str(x).split("-")))

item_mock_df = item_mock_df.drop('item_category-item_name', axis=1)

In [35]:
item_mock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   item_condition                  1500 non-null   object
 1   item_postcode                   1500 non-null   object
 2   item_status                     1500 non-null   object
 3   item_lat                        1500 non-null   object
 4   item_lng                        1500 non-null   object
 5   userwhochangeditemstatus_id     1500 non-null   object
 6   item_datetime_posted            1500 non-null   object
 7   item_datetimechangeditemstatus  1500 non-null   object
 8   item_id                         1500 non-null   int64 
 9   item_category                   1500 non-null   object
 10  item_name                       1500 non-null   object
dtypes: int64(1), object(10)
memory usage: 140.6+ KB


### To the Model

In [36]:
# define a function to create model data


def create_model_data(num_records): 
  
    # dictionary 
    model ={} 
    for i in range(0, num_records): 
        model[i] = {} 
        model[i]['item_id'] = np.random.choice(item_id_series)
        model[i]['item_category-item_name'] = np.random.choice(category_item) 
        model[i]['item_condition'] = np.random.choice(item_condition)
        model[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
        model[i]['item_lat'] = np.random.choice(list_lat)
        model[i]['item_lng'] = np.random.choice(list_lng)
        model[i]['item_status'] = np.random.choice(item_status)
        model[i]['searched_item_name-searched_item_category'] = np.random.choice(category_item)
        model[i]['searched_item_postcode'] = np.random.choice(postcodes_berlin_series)
        
        datetime_iteration1 = fake.date_between_dates(limit,'now')
        datetime_iteration2 = fake.date_between_dates(limit,'now')
        if datetime_iteration1 <= datetime_iteration2:
            model[i]['item_datetime_posted'] = datetime_iteration1
            model[i]['item_datetimechangeditemstatus'] = datetime_iteration2
        else:
            model[i]['item_datetime_posted'] = datetime_iteration2
            model[i]['item_datetimechangeditemstatus'] = datetime_iteration1  
            
        model[i]['n_likes'] = np.random.choice(n_likes)
        
        model[i]['user_looker_id'] = np.random.choice(user_id_series)
        model[i]['user_looker_lat'] = np.random.choice(list_lat)
        model[i]['user_looker_lng'] = np.random.choice(list_lng)
        
    return model


In [37]:
model_mock_df = pd.DataFrame(create_model_data(1500)).transpose()
# model_mock_df.head(5)
model_mock_df.shape

(1500, 15)

In [38]:
model_mock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 15 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   item_id                                    1500 non-null   object
 1   item_category-item_name                    1500 non-null   object
 2   item_condition                             1500 non-null   object
 3   item_postcode                              1500 non-null   object
 4   item_lat                                   1500 non-null   object
 5   item_lng                                   1500 non-null   object
 6   item_status                                1500 non-null   object
 7   searched_item_name-searched_item_category  1500 non-null   object
 8   searched_item_postcode                     1500 non-null   object
 9   item_datetime_posted                       1500 non-null   object
 10  item_datetimechangeditemstatus      

### Split columns of dependent variables

In [39]:
model_mock_df[['item_category','item_name']] = model_mock_df['item_category-item_name'].apply(lambda x: pd.Series(str(x).split("-")))
model_mock_df[['searched_item_name','searched_item_category']] = model_mock_df['searched_item_name-searched_item_category'].apply(lambda x: pd.Series(str(x).split("-")))

model_mock_df = model_mock_df.drop('item_category-item_name', axis=1)
model_mock_df = model_mock_df.drop('searched_item_name-searched_item_category', axis=1)

        

In [40]:
model_mock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   item_id                         1500 non-null   object
 1   item_condition                  1500 non-null   object
 2   item_postcode                   1500 non-null   object
 3   item_lat                        1500 non-null   object
 4   item_lng                        1500 non-null   object
 5   item_status                     1500 non-null   object
 6   searched_item_postcode          1500 non-null   object
 7   item_datetime_posted            1500 non-null   object
 8   item_datetimechangeditemstatus  1500 non-null   object
 9   n_stars                         1500 non-null   object
 10  user_looker_id                  1500 non-null   object
 11  user_looker_lat                 1500 non-null   object
 12  user_looker_lng                 1500 non-null   

In [41]:
model_mock_df.head()

,item_id,item_condition,item_postcode,item_lat,item_lng,item_status,searched_item_postcode,item_datetime_posted,item_datetimechangeditemstatus,n_stars,user_looker_id,user_looker_lat,user_looker_lng,item_category,item_name,searched_item_name,searched_item_category
0,458,1,10997,5.248.335.794,1.352.905.137,not_available,10317,2023-02-17,2023-02-18,3,706,5.244.015.683,1.336.432.302,tech,keyboard,appliance,vacuum_cleaner
1,334,1,10829,524.768.847,1.346.318.315,not_available,12247,2023-02-18,2023-03-13,3,498,5.249.558.027,1.356.921.324,lighting,lighting,appliance,heater
2,773,3,13355,5.250.914.057,1.334.844.761,not_available,10405,2023-02-24,2023-03-11,1,279,5.247.056.723,1.345.241.784,furniture,sofa,furniture,table
3,254,1,10967,5.262.464.481,1.326.116.302,avaliable,13403,2023-03-07,2023-03-15,4,1481,524.914.937,1.346.145.126,furniture,cupboard,miscelaneaous,carpet
4,773,3,12679,5.253.049.413,1.337.073.656,avaliable,10719,2023-02-21,2023-02-27,5,868,5.246.958.581,1.345.706.342,home,mattress,clothes,man_clothes


### Creating csv files

In [42]:
# user_mock_df.to_csv('data/user_mock_data.csv', index=False)

In [43]:
# item_mock_df.to_csv('data/item_mock_data.csv', index=False)

In [44]:
model_mock_df.to_csv('data/model_mock_data.csv', index=False)

## References:
- [Generate custom datasets using Python Faker](https://blogs.sap.com/2021/05/26/generate-custom-datasets-using-python-faker/)
- [folium_polygon_rectangle_layers](https://www.youtube.com/watch?v=9E9FTJrOJ1E&t=752s)
- [Faker](https://github.com/joke2k/faker/issues/1183)
- [Generating Mock Data with Python! (NumPy, Pandas, & Datetime Libraries)](https://www.youtube.com/watch?v=VJBY2eVtf7o)